Написать приложение и функцию, которые собирают основные новости с сайта на выбор dzen.ru, lenta.ru, mail.ru . Для парсинга использовать XPath
Структура данных должна содержать:
* название источника
* наименование новости
* ссылку на новость
* дата публикации

минимум один сайт максимум все

Задание считается выполненным, если:
- код задокументирован, парсер работает корректно
- репозиторий в github имеет readme.md с описанием и структурой проекта (если сдача github, gitlab )
- есть скриншоты результата выполнения задания

ДЗ принимается ссылками на github, gitlab, colab

## Подключение необходимых библиотек

In [1]:
import pandas as pd
import requests
from lxml import html
import os

## Объявление и инициализация констант

In [40]:
REQUIRED_SITE_NEWS_DZEN_URL = "https://lenta.ru"
REQUIRE_HEADER = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/110.0"
}

In [41]:
def get_page_content(url, header):
    '''
    Формирование контента страницы в виде строки
    '''
    page_data = requests.get(url, header)
    
    if not page_data.ok:
        raise Exception(page_data.status_code)
    
    return page_data.text

In [42]:
def get_all_records(dom_obj):
    'Формирование списка всех важных новостей'
    return dom_obj.xpath("//div[@class='last24']/a")

In [43]:
def get_links(all_records: list):
    '''
    Поиск ссылок на новости
    '''
    links = []
    for record in all_records:
        links.append(record.xpath('@href')[0])
    return links

In [44]:
def get_dates(links: list, url_site: str):
    '''
    Определение дат публикации
    '''
    dates_public = []
    
    for link in links:
        dates_public.append('-'.join(link.split(url_site)[-1].split('/')[2:5]))
    
    return dates_public

In [45]:
def get_news_title(all_records):
    '''
    Определение заголовков новостей
    '''

    news_title = list(
        map(
            lambda news: news.xpath("./div[@class='card-mini__text']/span[@class='card-mini__title']")[0].text,
            all_records
        )
    )
    return news_title
    
    

In [46]:
def get_parsing_result_table(page_dom, url_site_news) -> pd.DataFrame:
    all_records = get_all_records(page_dom)
    
    '''
    Формирование ссылок на новости
    '''
    links = list(map(lambda link: f"{REQUIRED_SITE_NEWS_DZEN_URL}{link}", get_links(all_records)))
    
    '''
    Поскольку на странице нет компоненита, содержащего
    дату, получим ее из ссылок
    '''
    dates_public = get_dates(links, url_site_news)
    
    '''
    Извлечение заголовков новостей
    '''
    news_title = get_news_title(all_records)
    
    return pd.DataFrame(
        {
            "Наименование новости": news_title,
            "Ссылка на новость": links,
            "Дата публикации": dates_public,
        }
    )
    

## Представление страницы в виде строки

In [47]:
page_data = get_page_content(REQUIRED_SITE_NEWS_DZEN_URL, REQUIRE_HEADER)

## Преобразование полученной строки к DOM-объекту

In [48]:
page_dom = html.fromstring(page_data)

## Поиск требуемых данных

In [49]:
parse_result = get_parsing_result_table(
    page_dom,
    REQUIRED_SITE_NEWS_DZEN_URL
)

In [50]:
parse_result

,Наименование новости,Ссылка на новость,Дата публикации
0,Врачи озвучили версии развития тяжелого состоя...,https://lenta.ru/news/2023/02/18/vrachi/,2023-02-18
1,Журналист Херш усомнился в признании США вины ...,https://lenta.ru/news/2023/02/18/nord_stream/,2023-02-18
2,В МИД ответили на призыв Китая прекратить огон...,https://lenta.ru/news/2023/02/18/zaxarova_ukra...,2023-02-18
3,Раскрыты подробности о заседании Совбеза ООН п...,https://lenta.ru/news/2023/02/18/hersh/,2023-02-18
4,Американский офицер предрек Зеленскому судьбу ...,https://lenta.ru/news/2023/02/17/hitler/,2023-02-17
5,Зеленский заверил в скором снятии табу на пост...,https://lenta.ru/news/2023/02/17/missiles/,2023-02-17
6,Путин разрешил купить долю европейского нефтян...,https://lenta.ru/news/2023/02/17/putin_shell/,2023-02-17
7,Французы раскритиковали слова Макрона о невозм...,https://lenta.ru/news/2023/02/17/francuzi/,2023-02-17
8,Спецназ в Москве задержал удерживавшего заложн...,https://lenta.ru/news/2023/02/17/moskk_/,2023-02-17
9,В России за полгода сменили командующих четыре...,https://lenta.ru/news/2023/02/17/voennye_okruga/,2023-02-17
